## Immobiliare.it web-scraping

In [55]:
# IMPORTING LIBRARIES

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [56]:
# INITIALIZING ITALIAN REGIONS AND TYPE OF ASSET

regioni = ['abruzzo', 'basilicata', 'campania', 'calabria', 'emilia-romagna', 'friuli-venezia-giulia', 'lazio','liguria', 'lombardia','marche','molise','piemonte','puglia','sardegna','sicilia','toscana', 'trentino-alto-adige','umbria','valle-d-aosta','veneto']
tipo = 'case'

In [57]:
# INITIALIZE THE VARIABLE LISTS

price_list = []
adv_list = []
region = []
locali = []
superficie = []
bagni = []

In [58]:
# FUNCTIONS TO GET ASSET DATA FROM TILES

def get_price(soup, price_list):
     price_list.append(soup.find('div', {"class": "in-realEstateListCard__priceOnTop"}).text)



def get_adv(soup, adv_list):
     adv_list.append(soup.find('a', class_="in-card__title").attrs.get('title'))



def get_locali(soup, locali_list):
     if soup.find('li', {"class" : "nd-list__item in-feat__item", "aria-label": "locali"}) is not None:
          locali_list.append(soup.find('li', {"class" : "nd-list__item in-feat__item", "aria-label": "locali"}).text)
     else:
          locali_list.append(np.nan)



def get_superficie(soup,superficie_list):
     if soup.find('li', {"class" : "nd-list__item in-feat__item", "aria-label": "superficie"}) is not None:
          superficie_list.append(soup.find('li', {"class" : "nd-list__item in-feat__item", "aria-label": "superficie"}).text)
     else:
          superficie_list.append(np.nan)



def get_bagni(soup, bagni_list):
     if soup.find('li', {"class" : "nd-list__item in-feat__item", "aria-label": "bagni"}) is not None:
          bagni_list.append(soup.find('li', {"class" : "nd-list__item in-feat__item", "aria-label": "bagni"}).text)
     else:
          bagni_list.append(1)

In [59]:
for reg in regioni:
    
    max_num_pages = 80

    root = f"https://www.immobiliare.it/vendita-{tipo}/{reg}/?criterio=rilevanza&pag="
        
    
    for i in range(1,81):
        
        url = f"{root}{i}"
        print(f"Sto esaminando:\t{url}")
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        house_card = soup.find_all('div', class_="nd-mediaObject__content in-card__content in-realEstateListCard__content")
        
        for _ in house_card:

            #Get the price
            get_price(_,price_list)

            #Get the description
            get_adv(_,adv_list)
            
            #Get the region
            region.append(reg)
            
            #Get how many rooms
            get_locali(_,locali)

            #Get the area
            get_superficie(_,superficie)

            #Get how many toilets
            get_bagni(_,bagni)

Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=1
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=2
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=3
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=4
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=5
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=6
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=7
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=8
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=9
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=10
Sto esaminando:	https://www.immobiliare.it/vendita-case/abruzzo/?criterio=rilevanza&pag=11
Sto esam

In [60]:
df = pd.DataFrame(
    {'advertisement': adv_list,
     'region': region,
     'rooms': locali,
     'toilets': bagni,
     'area': superficie,
     'price': price_list
    })

df.sample(10)

,advertisement,region,rooms,toilets,area,price
32535,"Mansarda via Milano, 3b, Castello Di Fiemme, C...",trentino-alto-adige,5,2,171m²,€ 450.000
22857,"Bilocale via Conte di Roccavione 107, Borgo Vi...",piemonte,2,1,46m²,€ 45.000
22621,"Villa bifamiliare Strada Tetti Vigna 4, Savigl...",piemonte,5+,2,385m²,€ 299.000
33383,"Appartamento da ristrutturare, primo piano, Vi...",trentino-alto-adige,5,2,138m²,€ 295.000
16651,"Trilocale via Averardo Buschi 12, Città Studi,...",lombardia,3,1,100m²,€ 419.000
11549,"Terratetto unifamiliare via della Chiesa 25, S...",friuli-venezia-giulia,4,1,320m²,€ 58.000
36676,"Trilocale Località Planpincieux 8, Plampincieu...",valle-d-aosta,3,1,72m²,€ 585.000
8750,"Bilocale viale Benvenuto Cellini 30, Valverde,...",emilia-romagna,2,1,39m²,€ 136.000
34499,"Villa unifamiliare via del Grecale 23, Casagli...",umbria,5+,3,295m²,€ 299.000
2052,"Rustico Santa Caterina, Santa Caterina, Maratea",basilicata,3,1,70m²,€ 129.000


In [61]:
df['city'] = df["advertisement"].apply(lambda x: x.split(',')[-1])


In [62]:
df = df[['region','city','area','rooms','toilets','price']]
df.sample(10)

,region,city,area,rooms,toilets,price
5940,campania,Agropoli,45m²,2,1,€ 55.000
22838,piemonte,Torino,50m²,2,1,€ 195.000
26597,sardegna,Olbia,185m²,5,3,€ 550.000
10666,friuli-venezia-giulia,Spilimbergo,150m²,5+,2,€ 150.000
6583,calabria,Chiaravalle Centrale,102m²,3,1,€ 40.000
27384,sardegna,Sassari,95m²,5+,1,€ 60.000
19661,marche,Ancona,114m²,3,2,€ 110.000
38182,veneto,Castelfranco Veneto,169m²,4,2,€ 379.000
28160,sicilia,San Giovanni la Punta,224m²,5,2,€ 199.500
36127,valle-d-aosta,Brusson,100m²,3,1,€ 275.000


In [63]:
df.to_csv('house_prices_italy.csv')